In [1]:
import sys
sys.path.append('../')

import tquant as tq
from engine.globalconfig import GlobalConfig
from engine.loader.curveload import MarketDataLoader
from engine.loader.inflationindexload import Inflation_Index

from datetime import date, timedelta
import tensorflow as tf
import pandas as pd

import QuantLib as ql

# Settings

In [2]:
config = GlobalConfig.from_json()
mdl = MarketDataLoader(config)
tq.Settings.evaluation_date

datetime.date(2023, 9, 29)

In [3]:
calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

# Market

In [4]:
rates = mdl.ir_eur_curve_estr['quote'].values
times = mdl.ir_eur_curve_estr['daycount'].values/365

rates_infl = [0.1,0.2,0.2,0.22,0.24,0.23,0.25,0.26,0.28,0.3,0.24,0.15,0.12,0.10,0.11,0.09,0.05,0.4,0.004,0.1]
times_infl = [0,1,10,100,180,250,350,365,380,450,540,600,700,850,950,1000,1100,1200,1400,2000]

In [5]:
estr_curve = tq.RateCurve(times, rates)
infl_curve = tq.InflationCurveSimple(times_infl,rates_infl)

# Market Quantlib

In [6]:
today = ql.Date(29,9,2023)
ql.Settings.instance().evaluationDate = today

ql_times = [today + int(day*365) for day in times]

ql_times_infl = [today + int(day) for day in times_infl]

In [7]:
ql_estr_curve = ql.ZeroCurve(ql_times,rates,ql.Actual360())
ql_estr_handle = ql.RelinkableYieldTermStructureHandle()
ql_estr_handle.linkTo(ql_estr_curve)


observationLag = ql.Period("0M")
ql_infl_curve = ql.ZeroInflationCurve(ql.Date(1,9,2023), ql.TARGET(), ql.Actual360(),observationLag,ql.Monthly,ql_times_infl,rates_infl)
ql_infl_handle = ql.RelinkableZeroInflationTermStructureHandle()
ql_infl_handle.linkTo(ql_infl_curve)

# Inflation Coupon Quantlib

In [8]:
# Indici di inflazione molto complessi. Bisogna capire bene come fanno - C'era l'indice EUHCPI per cui non funzionava questo stesso codice
# Discutere che l'indice può essere fisso alla prima data del mese
swapType = ql.ZeroCouponInflationSwap.Payer
calendar = ql.TARGET()
nominal = 1e6
startDate = ql.Date(1,11,2025)
endDate = ql.Date(1,11,2026)
fixedRate = 0.1
dc = ql.Actual365Fixed()

inflationIndex = ql.UKRPI(False,ql_infl_handle)
inflationIndex.addFixing(ql.Date(29,9,2023), 0.1, True)

contractObservationLag = ql.Period(3, ql.Months)
bdc = ql.ModifiedFollowing
swap = ql.ZeroCouponInflationSwap(swapType, nominal, startDate, endDate, calendar, bdc, dc, fixedRate, inflationIndex, contractObservationLag, ql.CPI.Linear)

swapEngine = ql.DiscountingSwapEngine(ql_estr_handle)
swap.setPricingEngine(swapEngine)
npv = swap.NPV()
print("QuantLib ZeroCoupon NPV:", npv)

QuantLib ZeroCoupon NPV: 108330.07593669387


# Inflation Coupon TQuant

In [9]:
swapType = tq.SwapType.Payer
calendar = tq.TARGET()
nominal = 1e6
startDate = tq.date(2025,11,1)
endDate = tq.date(2026,11,1)
fixedRate = 0.1
dc = tq.DayCounter(tq.DayCounterConvention.Actual365)
payment_lag = 3
payment_lag_period = tq.TimeUnit.Months

inflationIndex = Inflation_Index.euhicp
inflationIndex.add_fixing(tq.date(2023,9,29), 0.1)

contractObservationLag = 3
contractObservationLagTenor = tq.TimeUnit.Months

bdc = tq.BusinessDayConvention.ModifiedFollowing
coupon = tq.ZeroInflationCoupon(endDate, nominal, startDate, endDate, inflationIndex, fixedRate, swapType, startDate, endDate, dc, payment_lag, payment_lag_period)

coupon_engine = tq.ZeroInflationCouponDiscounting(coupon, False)
npv_coupon = coupon_engine.price(estr_curve, infl_curve, evaluation_date)

print("TQuant Zero Coupon NPV:", npv_coupon.numpy())

TQuant Zero Coupon NPV: 108226.3266877565


# YoY Inflation Swap Quantlib

In [10]:
observationLag = ql.Period("0M")
ql_infl_curve = ql.YoYInflationCurve(ql.Date(1,9,2023), ql.TARGET(), ql.Actual360(),observationLag,ql.Monthly,True,ql_times_infl,rates_infl)
ql_infl_curve.enableExtrapolation()
ql_infl_handle = ql.RelinkableYoYInflationTermStructureHandle()
ql_infl_handle.linkTo(ql_infl_curve)

swapType = ql.YearOnYearInflationSwap.Payer
nominal = 1e6
startDate = ql.Date(1,11,2025)
endDate = ql.Date(1,11,2026)

fixedSchedule = ql.MakeSchedule(startDate, endDate, ql.Period('6m'))
fixedRate = 0.1
fixedDayCounter = ql.Actual365Fixed()
yoySchedule = ql.MakeSchedule(startDate, endDate, ql.Period('6m'))
index = ql.YYEUHICP(False,ql_infl_handle)
lag = ql.Period('3m')
spread = 0.0
yoyDayCounter = ql.Actual365Fixed()
paymentCalendar = ql.TARGET()

swap = ql.YearOnYearInflationSwap(swapType, 
                                  nominal, 
                                  fixedSchedule, 
                                  fixedRate, 
                                  fixedDayCounter, 
                                  yoySchedule, 
                                  index, 
                                  lag, 
                                  spread, 
                                  yoyDayCounter, 
                                  paymentCalendar)

swapEngine = ql.DiscountingSwapEngine(ql_estr_handle)
swap.setPricingEngine(swapEngine)
npv = swap.NPV()
print("Quantlib YoYinflationSwap NPV:", npv)
#print(swap.fixedLegNPV())
#print(swap.yoyLegNPV())

Quantlib YoYinflationSwap NPV: -11011.224008716905


# YoY Inflation Swap TQuant

In [11]:
infl_curve = tq.YoYInflationCurveSimple(times_infl,rates_infl)

swapType = tq.SwapType.Payer
startDate = tq.date(2025,11,1)
endDate = tq.date(2026,11,1)

fixed_schedule = [startDate]
fixed_schedule.extend([startDate + timedelta(365 *.5 *i) for i in range(1,3)])
fixedDayCounter = tq.DayCounter(tq.DayCounterConvention.Actual365)

nominal = [1e6]*len(fixed_schedule)
fixedRate = [0.1]*len(fixed_schedule)

yoySchedule = fixed_schedule
inflationIndex = Inflation_Index.euhicp
inflationIndex.add_fixing(tq.date(2023,9,29), 0.1)
contractObservationLag = 3
contractObservationLagTenor = tq.TimeUnit.Months
spread = 0.0
yoyDayCounter = tq.DayCounter(tq.DayCounterConvention.Actual365)

paymentCalendar = tq.TARGET()

leg1 = tq.FixedRateLeg(fixed_schedule,nominal,fixedRate, fixedDayCounter)

leg2 = tq.InflationLeg(yoySchedule,nominal,inflationIndex,yoyDayCounter, spread = spread, observation_lag=contractObservationLag, observation_lag_period=contractObservationLagTenor)

swap = tq.InflationSwap.from_legs(swapType,leg1,leg2)

swap_engine = tq.InflationSwapAnalyticEngine(swap)
npv_swap = swap_engine.price(estr_curve, infl_curve, evaluation_date)

print("TQuant YoYinflation Swap NPV:",npv_swap.numpy())

TQuant YoYinflation Swap NPV: -10948.52255107733


In [12]:
# Per ora l'errore deriva dal discount. Controllare dopo l'inserimento delle curve
print(fixed_schedule)
for date in fixedSchedule:
    print(date)
print(estr_curve.discount(2.591780821917808))
print(ql_estr_curve.discount(ql.Date(2,5,2026)))

[datetime.date(2025, 11, 1), datetime.date(2026, 5, 2), datetime.date(2026, 11, 1)]
November 1st, 2025
May 1st, 2026
November 1st, 2026
tf.Tensor(0.9152458412443154, shape=(), dtype=float64)
0.9142942981177891
